In [ ]:
import os
import numpy
import pandas as pd

In [ ]:
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1

In [ ]:
mgh_mrns = list(map(lambda filename: filename[:-4], os.listdir('/storage/shared/ecg/mgh')))
bwh_mrns = list(map(lambda filename: filename[:-4], os.listdir('/storage/shared/ecg/bwh')))

if 'bad_mrn' in mgh_mrns:
    mgh_mrns.remove('bad_mrn')
if 'bad_mrn' in bwh_mrns:
    bwh_mrns.remove('bad_mrn')

mgh_mrns = list(map(int, mgh_mrns))
bwh_mrns = list(map(int, bwh_mrns))

ecg_mrns = set(mgh_mrns) | set(bwh_mrns)

In [ ]:
c3po = pd.read_csv('~/dropbox/c3po-data/c3po-formatted-mrn.csv')
c3po_mrns = set(c3po['mrn'])

In [ ]:
# echo = set(pd.read_csv('~/dropbox/aortic-stenosis-data/echo.csv')['mrn'])
sts = set(pd.read_csv('~/dropbox/sts-data/mgh.csv')['medrecn'])
# ici = set(pd.read_csv('~/dropbox/ici-data/ici.csv')['mgh_mrn'])
mrns_to_remove = sts # echo | sts | ici
print(f"mrns_to_remove: {len(mrns_to_remove)}")

In [ ]:
overlapped = set(mgh_mrns) & set(bwh_mrns)

In [ ]:
overlapped

In [ ]:
mrns = (c3po_mrns & ecg_mrns) - mrns_to_remove
print(f"mrns: {len(mrns)}")

In [ ]:
c3po = c3po[c3po['mrn'].isin(mrns)]

In [ ]:
c3po_lived = c3po[c3po['death_date'].isnull()]
c3po_died = c3po[~c3po['death_date'].isnull()]

In [ ]:
c3po.sample(frac=1, random_state=0)

l_tv_idx = int(train_ratio * len(c3po_lived))
l_vt_idx = int((train_ratio + valid_ratio) * len(c3po_lived))

d_tv_idx = int(train_ratio * len(c3po_died))
d_vt_idx = int((train_ratio + valid_ratio) * len(c3po_died))

train_lived = c3po_lived[:l_tv_idx]
valid_lived = c3po_lived[l_tv_idx:l_vt_idx]
test_lived = c3po_lived[l_vt_idx:]

train_died = c3po_died[:d_tv_idx]
valid_died = c3po_died[d_tv_idx:d_vt_idx]
test_died = c3po_died[d_vt_idx:]

In [ ]:
# Create splits for varying percentile amounts of c3po cohort
for percent in [100]: #, 90, 75, 50, 25, 10]:
    print(f"Parsing c3po data for {percent} percent")
    suffix = ""
    if percent != 100:
        suffix = f"-{percent}"
    frac = percent / 100
    _train_lived = train_lived.sample(frac=frac, random_state=percent)
    _valid_lived = valid_lived.sample(frac=frac, random_state=percent)
    _test_lived = test_lived.sample(frac=frac, random_state=percent)
    _train_died = train_died.sample(frac=frac, random_state=percent)
    _valid_died = valid_died.sample(frac=frac, random_state=percent)
    _test_died = test_died.sample(frac=frac, random_state=percent)

    train = pd.concat([_train_lived, _train_died]).sample(frac=1, random_state=percent)
    valid = pd.concat([_valid_lived, _valid_died]).sample(frac=1, random_state=percent)
    test = pd.concat([_test_lived, _test_died]).sample(frac=1, random_state=percent)
    
    train['mrn'].to_csv(f'~/dropbox/c3po-data/train{suffix}.csv', index=False)
    valid['mrn'].to_csv(f'~/dropbox/c3po-data/valid{suffix}.csv', index=False)
    test['mrn'].to_csv(f'~/dropbox/c3po-data/test{suffix}.csv', index=False)